In [52]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from os import getcwd
from UliPlot.XLSX import auto_adjust_xlsx_column_width
from datetime import datetime

In [53]:
pmsystem_project_list_file = '(Input) ProjectListExport.xls'
cft_list_file = '(Input) CFTListExport.xls'
prweb_file = '(Input) New Production Status Summary.xlsx'

start_from = '2022-05'

consumer_dpqe_list = ['Lo, Aalto','Kuo, Chris','Wang, Cooper', 'Fu, Ernest','Hsu, Jason(PQE)','Ni, Wendy','Wang, Evelyn','Hsu, Jason(PQE);\nWang, Evelyn'] 
# 'Hsu, Jason(PQE);\nWang, Evelyn' is for GPSMAP 9000 series

# PM system

Read

In [54]:
pmsystem_project = pd.read_excel(pmsystem_project_list_file, usecols=[0,1,3,4,5,6,9,10,12], parse_dates=[2,3,4,5], index_col=[0])
pmsystem_cft = pd.read_excel(cft_list_file, usecols=[0,44], index_col=[0])

c:\Users\wangcooper\Miniconda3\envs\workspace\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [55]:
pmsystem_cft.head()

,Develop Product QE (DPQE)
Project ID,
15624,"Lo, Aalto"
13009,"Kuo, Chris"
14984,"Kuo, Chris"
16199,"Ni, Wendy"
16547,NaN


In [56]:
pmsystem_project = pmsystem_project[(pmsystem_project['PDP/Form107'] == 'PDP')]

In [57]:
pmsystem_project.head()

,Project Name,Working Prototype Built and Tested,Final Factory Prototypes Built,MP Date,PR Date,Global/Asia,PDP/Form107,Status
Project ID,,,,,,,,
15624,Catalyst Kart/Moto/Roadster,2022-06-24,NaT,2023-03-20,2023-01-09,Global,PDP,Development
13009,Dash Cam Live (LTE),2021-06-25,2022-03-11,2022-10-24,2022-08-24,Global,PDP,Development
14984,"Dash Cam Live (LTE), ASIA",NaT,2022-03-31,2022-11-07,2022-10-17,Asia,PDP,Development
16199,DC-2023 Premium Dash Cameras,2022-12-22,2023-05-10,2023-07-31,2023-05-31,Global,PDP,Development
16547,"DC-2023 Premium Dash Cameras, Asia",NaT,NaT,NaT,NaT,Asia,PDP,Development


Merge project and CFT lists

In [58]:
pmsystem_project_cft = pd.merge(left=pmsystem_project, right=pmsystem_cft, left_index=True, right_index=True)
pmsystem_project_cft.sort_index().head()

,Project Name,Working Prototype Built and Tested,Final Factory Prototypes Built,MP Date,PR Date,Global/Asia,PDP/Form107,Status,Develop Product QE (DPQE)
Project ID,,,,,,,,,
2854,Product - GDC 7500,2013-04-30,NaT,2019-07-01,2018-10-12,Global,PDP,Development,"Kuo, Chris"
2914,Product - GTN 6xx/7xx,NaT,NaT,NaT,NaT,Global,PDP,Development,NaN
2930,Product - GWX 7000,2019-03-22,NaT,2021-09-09,2021-06-04,Global,PDP,Concepts,NaN
3753,Product - GDU 7XX,2015-08-14,NaT,2018-01-26,2017-01-05,Global,PDP,Development,NaN
3767,HIRF & Lightning System Cert,NaT,NaT,NaT,NaT,Global,PDP,Development,NaN


Find projects which WW/ASIA SKU have different DPQE

In [59]:
ww_asia_diff = pmsystem_project_cft.copy()

In [60]:
#Handle Asis words in 'Project Name' column
ww_asia_diff['Project Name'] = ww_asia_diff['Project Name'].apply(lambda name : name.split(',')[0])
ww_asia_diff['Project Name'] = ww_asia_diff['Project Name'].str.replace('ASIA','')
ww_asia_diff['Project Name'] = ww_asia_diff['Project Name'].str.replace('Asia','')

In [61]:
# check any missing
ww_asia_diff[ww_asia_diff['Project Name'].str.contains('ASIA|asia|Asia')]

,Project Name,Working Prototype Built and Tested,Final Factory Prototypes Built,MP Date,PR Date,Global/Asia,PDP/Form107,Status,Develop Product QE (DPQE)
Project ID,,,,,,,,,


In [62]:
ww_asia_diff = ww_asia_diff[['Project Name','Develop  Product QE (DPQE)','Global/Asia','MP Date']].loc[ww_asia_diff['MP Date']> start_from]

In [63]:
# Find duplicated projects
ww_asia_diff = ww_asia_diff[ww_asia_diff.duplicated(subset=['Project Name'],keep=False)]
ww_asia_diff

,Project Name,Develop Product QE (DPQE),Global/Asia,MP Date
Project ID,,,,
13009,Dash Cam Live (LTE),"Kuo, Chris",Global,2022-10-24
14984,Dash Cam Live (LTE),"Kuo, Chris",Asia,2022-11-07
14994,Edge Explore 2,"Kuo, Chris",Global,2022-05-30
16259,Edge Explore 2,"Kuo, Chris",Asia,2022-07-25
15993,Lily 2,"Fu, Ernest",Global,2023-10-30
16414,Lily 2,"Fu, Ernest",Asia,2023-10-16
15520,Venu 3,"Wang, Cooper",Global,2023-05-29
16260,Venu 3,"Wang, Cooper",Asia,2023-05-29
15519,Venu 3S,"Fu, Ernest",Global,2023-05-29


In [64]:
# Find projects have different DPQE
ww_asia_diff_DPQE = ww_asia_diff[~ww_asia_diff.duplicated(subset=['Project Name','Develop  Product QE (DPQE)'],keep=False)]

Find projects w/o DPQE (include ASIA SKUs)

In [65]:
projects_wo_DPQE = pmsystem_project_cft[pmsystem_project_cft['Develop  Product QE (DPQE)'].isnull()].dropna(thresh=5, axis=0).loc[(pmsystem_project_cft['MP Date']>start_from)]

Find projects with multi-DPQE (include ASIA SKUs)

In [66]:
multi_dpqe = pmsystem_project_cft[['Project Name','Global/Asia','Develop  Product QE (DPQE)','MP Date']]
multi_dpqe = multi_dpqe[multi_dpqe['MP Date']>start_from]
multi_dpqe.dropna(inplace=True)

In [67]:
multi_dpqe = multi_dpqe[multi_dpqe['Develop  Product QE (DPQE)'].str.contains('\n')]

Melt

In [68]:
df_consumer_dpqe = pmsystem_project_cft[pmsystem_project_cft['Develop  Product QE (DPQE)'].isin(consumer_dpqe_list)].sort_index()
df_consumer_dpqe = df_consumer_dpqe[df_consumer_dpqe['Global/Asia']=='Global']
df_consumer_dpqe.head()

,Project Name,Working Prototype Built and Tested,Final Factory Prototypes Built,MP Date,PR Date,Global/Asia,PDP/Form107,Status,Develop Product QE (DPQE)
Project ID,,,,,,,,,
2854,Product - GDC 7500,2013-04-30,NaT,2019-07-01,2018-10-12,Global,PDP,Development,"Kuo, Chris"
5711,Product - GLA 85,2014-10-30,NaT,2018-12-07,2016-08-31,Global,PDP,Development,"Kuo, Chris"
7011,Xero G2i SatCom Scouting Camera,2020-03-03,2020-11-09,NaT,2021-08-27,Global,PDP,Development,"Kuo, Chris"
7029,Alpha Dog Collar,2021-11-18,2022-08-18,2023-01-03,2022-10-28,Global,PDP,Development,"Kuo, Chris"
7563,Mercury,NaT,NaT,2022-01-17,2021-04-23,Global,PDP,Development,"Fu, Ernest"


In [69]:
value_vars = ['Working Prototype Built and Tested','Final Factory Prototypes Built','PR Date','MP Date']
pmsystem_project_dpqe_melted = df_consumer_dpqe.reset_index().melt(id_vars= ['Project ID','Project Name','Develop  Product QE (DPQE)'],value_vars=value_vars)
pmsystem_project_dpqe_melted.head()

,Project ID,Project Name,Develop Product QE (DPQE),variable,value
0,2854,Product - GDC 7500,"Kuo, Chris",Working Prototype Built and Tested,2013-04-30
1,5711,Product - GLA 85,"Kuo, Chris",Working Prototype Built and Tested,2014-10-30
2,7011,Xero G2i SatCom Scouting Camera,"Kuo, Chris",Working Prototype Built and Tested,2020-03-03
3,7029,Alpha Dog Collar,"Kuo, Chris",Working Prototype Built and Tested,2021-11-18
4,7563,Mercury,"Fu, Ernest",Working Prototype Built and Tested,NaT


In [70]:
pmsystem_project_dpqe_melted.rename(columns={'Develop  Product QE (DPQE)':'DPQE','variable':'Phase','value':'Date'},inplace=True)

In [71]:
pmsystem_project_dpqe_melted.Date = pmsystem_project_dpqe_melted.Date.dt.strftime('%Y-%m')

In [72]:
pmsystem_project_dpqe_melted = pmsystem_project_dpqe_melted[pmsystem_project_dpqe_melted.Date >= start_from]

In [73]:
pmsystem_project_dpqe_melted.isnull().sum()

Project ID      0
Project Name    0
DPQE            0
Phase           0
Date            0
dtype: int64

Weight socre

In [74]:
pmsystem_project_dpqe_melted.loc[:,'weighted_score'] = pmsystem_project_dpqe_melted.loc[:,'Phase'].apply(lambda phase : 1 if (phase == 'Working Prototype Built and Tested') | (phase == 'MP Date') else 2 if phase=='Final Factory Prototypes Built' else 3 if phase=='PR Date' else 0)

In [75]:
pmsystem_project_dpqe_melted = pmsystem_project_dpqe_melted.rename(columns={"Project ID":'PM Project Id','Project Name':'PM Project Name'})

In [76]:
pmsystem_project_dpqe_melted.head()

,PM Project Id,PM Project Name,DPQE,Phase,Date,weighted_score
6,8510,Xero L60i Laser Navigator,"Kuo, Chris",Working Prototype Built and Tested,2022-08,1
9,9946,GPSMAP 9000,"Hsu, Jason(PQE);\nWang, Evelyn",Working Prototype Built and Tested,2022-06,1
12,10617,Force Kayak Propulsion System,"Hsu, Jason(PQE)",Working Prototype Built and Tested,2023-03,1
15,10901,eTrex Solar,"Kuo, Chris",Working Prototype Built and Tested,2022-12,1
19,11778,Tire Temp Sensor,"Ni, Wendy",Working Prototype Built and Tested,2022-08,1


# PRweb

Read

In [77]:
prweb = pd.read_excel(prweb_file, header=1, usecols=[0,3])
prweb.head()

,Project Name,PM Project Id
0,Xero B60i Laser Navigation Binoculars,13229
1,TGWW25,16227
2,LYWW24,15976
3,Force Kayak Propulsion System,10617
4,LED Control Module,13134


Merge with data from PMsystem

In [78]:
prweb_pmsystem = pd.merge(left=pmsystem_project_dpqe_melted, right=prweb, how='left', left_on='PM Project Id', right_on='PM Project Id', suffixes=['_PMsystem','_prweb'])

In [79]:
prweb_pmsystem.head()

,PM Project Id,PM Project Name,DPQE,Phase,Date,weighted_score,Project Name
0,8510,Xero L60i Laser Navigator,"Kuo, Chris",Working Prototype Built and Tested,2022-08,1,Xero L60i Laser Navigator
1,9946,GPSMAP 9000,"Hsu, Jason(PQE);\nWang, Evelyn",Working Prototype Built and Tested,2022-06,1,GPSMAP 9000_Black Box
2,9946,GPSMAP 9000,"Hsu, Jason(PQE);\nWang, Evelyn",Working Prototype Built and Tested,2022-06,1,"GPSMAP 9000 - 19"""
3,9946,GPSMAP 9000,"Hsu, Jason(PQE);\nWang, Evelyn",Working Prototype Built and Tested,2022-06,1,"GPSMAP 9000 - 24"""
4,9946,GPSMAP 9000,"Hsu, Jason(PQE);\nWang, Evelyn",Working Prototype Built and Tested,2022-06,1,"GPSMAP 9000 -27"""


Handle GPSMAP 9000 19"

In [80]:
'Hsu, Jason(PQE);\nWang, Evelyn'.replace('\n','').split(';')

['Hsu, Jason(PQE)', 'Wang, Evelyn']

In [81]:
# GPSMAP 9000 - 19" to Wang, Evelyn, others to Hsu, Jason(PQE)
def gpsmap_9000 (project_name, DPQE):
    if project_name == 'GPSMAP 9000 - 19"':
        return DPQE.replace('\n','').split(';')[1] #Wang, Evelyn
    elif project_name in ['GPSMAP 9000 - 22"','GPSMAP 9000 - 24"','GPSMAP 9000 -27"','GPSMAP 9000_Black Box']:
        return DPQE.replace('\n','').split(';')[0] #Hsu, Jason(PQE)
    else:
        return DPQE


In [82]:
prweb_pmsystem['DPQE'] = np.vectorize(gpsmap_9000)(prweb_pmsystem['Project Name'],prweb_pmsystem['DPQE'])

In [83]:
#Validate
prweb_pmsystem[prweb_pmsystem['PM Project Name'] == 'GPSMAP 9000']

,PM Project Id,PM Project Name,DPQE,Phase,Date,weighted_score,Project Name
1,9946,GPSMAP 9000,"Hsu, Jason(PQE)",Working Prototype Built and Tested,2022-06,1,GPSMAP 9000_Black Box
2,9946,GPSMAP 9000,"Wang, Evelyn",Working Prototype Built and Tested,2022-06,1,"GPSMAP 9000 - 19"""
3,9946,GPSMAP 9000,"Hsu, Jason(PQE)",Working Prototype Built and Tested,2022-06,1,"GPSMAP 9000 - 24"""
4,9946,GPSMAP 9000,"Hsu, Jason(PQE)",Working Prototype Built and Tested,2022-06,1,"GPSMAP 9000 -27"""
5,9946,GPSMAP 9000,"Hsu, Jason(PQE)",Working Prototype Built and Tested,2022-06,1,"GPSMAP 9000 - 22"""
46,9946,GPSMAP 9000,"Hsu, Jason(PQE)",Final Factory Prototypes Built,2022-12,2,GPSMAP 9000_Black Box
47,9946,GPSMAP 9000,"Wang, Evelyn",Final Factory Prototypes Built,2022-12,2,"GPSMAP 9000 - 19"""
48,9946,GPSMAP 9000,"Hsu, Jason(PQE)",Final Factory Prototypes Built,2022-12,2,"GPSMAP 9000 - 24"""
49,9946,GPSMAP 9000,"Hsu, Jason(PQE)",Final Factory Prototypes Built,2022-12,2,"GPSMAP 9000 -27"""
50,9946,GPSMAP 9000,"Hsu, Jason(PQE)",Final Factory Prototypes Built,2022-12,2,"GPSMAP 9000 - 22"""


# Style DataFrame

In [84]:
detail = prweb_pmsystem.groupby(['DPQE','Project Name','Date']).agg({'weighted_score':'sum','Project Name':'count'}).sort_index(level='Date').rename(columns=({'weighted_score':'Weighted_Project_counts','Project Name':'Project_counts'})).unstack()

Overview

In [100]:
overview = prweb_pmsystem.groupby(['DPQE','Date']).count()['Project Name'].sort_index(level='Date').unstack(fill_value=0).style.background_gradient(cmap="OrRd")
overview

Date,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2024-01,2024-03,2024-04,2024-07,2024-09
DPQE,,,,,,,,,,,,,,,,,,,,,,,,
"Fu, Ernest",0,0,0,2,4,4,2,3,4,1,2,1,1,0,1,0,1,1,0,0,0,0,0,0
"Hsu, Jason(PQE)",0,8,3,1,1,6,3,11,3,2,5,5,1,5,1,0,0,1,0,1,0,1,1,0
"Kuo, Chris",4,2,2,3,3,5,2,3,2,0,3,1,1,2,2,2,2,0,0,0,1,1,2,1
"Lo, Aalto",0,4,0,3,2,3,3,2,4,3,4,0,0,0,1,0,0,0,0,0,0,0,0,0
"Ni, Wendy",1,0,0,3,2,1,0,3,1,3,1,1,2,2,1,0,0,0,0,0,0,0,0,0
"Wang, Cooper",0,0,4,2,2,5,1,3,4,1,4,1,2,0,1,1,1,1,0,0,0,0,0,0
"Wang, Evelyn",0,1,0,0,0,1,0,1,1,0,2,1,0,1,0,1,0,0,1,1,0,0,0,0


Overview_weighted

In [101]:
overview_weighted = prweb_pmsystem.groupby(['DPQE','Date']).sum()['weighted_score'].sort_index(level='Date').unstack(fill_value=0).style.background_gradient(cmap="OrRd")
overview_weighted

Date,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2024-01,2024-03,2024-04,2024-07,2024-09
DPQE,,,,,,,,,,,,,,,,,,,,,,,,
"Fu, Ernest",0,0,0,3,6,7,6,6,8,2,2,2,1,3,4,0,1,1,0,0,0,0,0,0
"Hsu, Jason(PQE)",0,11,3,1,2,12,12,21,8,2,8,15,1,8,4,0,0,1,0,2,0,3,1,0
"Kuo, Chris",12,3,3,7,5,11,2,5,2,0,6,2,1,6,4,2,2,0,0,0,2,3,4,1
"Lo, Aalto",0,5,0,5,3,4,6,5,10,7,4,0,0,0,1,0,0,0,0,0,0,0,0,0
"Ni, Wendy",1,0,0,5,4,3,0,5,1,9,1,1,5,2,1,0,0,0,0,0,0,0,0,0
"Wang, Cooper",0,0,4,5,3,9,1,6,12,1,4,2,3,0,3,3,1,1,0,0,0,0,0,0
"Wang, Evelyn",0,1,0,0,0,2,0,2,3,0,2,3,0,1,0,2,0,0,3,1,0,0,0,0


# 2022/09/02 request by Ray

 Find PM system projects MP dates from NOW

In [102]:
pmsystem_project2 = pd.read_excel(pmsystem_project_list_file, usecols=[0,1,5,8,9,10,12], 
    parse_dates=[2,3,4,5], engine='openpyxl')
pmsystem_project2 = pmsystem_project2[pmsystem_project2['Global/Asia']=='Global']
pmsystem_project2.head()

c:\Users\wangcooper\Miniconda3\envs\workspace\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Project ID,Project Name,MP Date,Category,Global/Asia,PDP/Form107,Status
0,15624,Catalyst Kart/Moto/Roadster,2023-03-20,Automotive,Global,PDP,Development
1,13009,Dash Cam Live (LTE),2022-10-24,Automotive,Global,PDP,Development
3,16199,DC-2023 Premium Dash Cameras,2023-07-31,Automotive,Global,PDP,Development
5,16467,Drive 53,2022-12-05,Automotive,Global,PDP,Development
6,11778,Tire Temp Sensor,2023-04-10,Automotive,Global,PDP,Development


In [103]:
MPdate_from_now = pmsystem_project2[pmsystem_project2['MP Date'] > datetime.now()].sort_values(['Category','MP Date'], ascending=True).set_index('Category')
MPdate_from_now['MP Date'] = MPdate_from_now['MP Date'].dt.date
MPdate_from_now

,Project ID,Project Name,MP Date,Global/Asia,PDP/Form107,Status
Category,,,,,,
Automotive,12983,Tread XL - Baja Race Edition,2022-10-17,Global,PDP,Development
Automotive,13009,Dash Cam Live (LTE),2022-10-24,Global,PDP,Development
Automotive,16467,Drive 53,2022-12-05,Global,PDP,Development
Automotive,15955,"Tread 8"" OEM - Honda",2023-01-09,Global,PDP,Development
Automotive,16069,zumo XT2,2023-03-06,Global,PDP,Development
...,...,...,...,...,...,...
Outdoor,16205,Descent Mk3s,2023-09-18,Global,PDP,Development
Outdoor,8510,Xero L60i Laser Navigator,2023-09-18,Global,PDP,Development
Outdoor,12994,Descent Mk3,2023-10-02,Global,PDP,Development


In [104]:
MPdate_from_now_count = MPdate_from_now.groupby('Category')['Project Name'].count().to_frame().rename({'Project Name':'Count'},axis=1)
MPdate_from_now_count

,Count
Category,
Automotive,8
Fitness,11
Marine,17
Outdoor,25


In [105]:
No_MPdate = pmsystem_project2[pmsystem_project2['MP Date'].isnull()].sort_values(['Category','MP Date'], ascending=True).set_index('Category')
No_MPdate = No_MPdate[~No_MPdate.index.isin(['Aviation','Garmin Labs'])]
No_MPdate

,Project ID,Project Name,MP Date,Global/Asia,PDP/Form107,Status
Category,,,,,,
Automotive,16319,Aviation Headset -,NaT,Global,PDP,Concepts
Automotive,16249,dezl headset (2nd Gen),NaT,Global,PDP,Concepts
Fitness,16437,Forerunner 165 and 165 Music,NaT,Global,PDP,Development
Fitness,16197,Edge 1050,NaT,Global,PDP,Concepts
Fitness,16345,Elevate Accessory,NaT,Global,PDP,Concepts
...,...,...,...,...,...,...
Outdoor,16037,M60i Laser Navigation Binoculars 15/18x56mm,NaT,Global,PDP,Concepts
Outdoor,16212,Marq (Gen 3),NaT,Global,PDP,Concepts
Outdoor,12279,Marq Hybrid,NaT,Global,PDP,Concepts


In [106]:
No_MPdate_count = No_MPdate.groupby('Category')['Project Name'].count().to_frame().rename({'Project Name':'Count'},axis=1)
No_MPdate_count

,Count
Category,
Automotive,2
Fitness,14
Marine,12
OEM Auto,20
Outdoor,25


Find Avation projects MP dates from PRweb

In [107]:
prweb2 = pd.read_excel(prweb_file, header=1, usecols=[0,1,5,6])
Aviation_PRweb = prweb2[prweb2['Category'] == 'Aviation'].sort_values('MP Date')
Aviation_PRweb

,Project Name,Project Id,Category,MP Date
17,Product - G3X/G500TXi Hybrid,1050,Aviation,2023-07-01
18,"Product - 1.3\"" GTR/GNC",984,Aviation,2023-07-01
5,Product - GDU147XW,1049,Aviation,2024-01-03
145,Product - Flight Stream Redesign,977,Aviation,NaN
146,Product - GDU77XW,1048,Aviation,NaN


# to_excel

In [108]:
with pd.ExcelWriter(getcwd()+'\(Output) Data.xlsx', engine='openpyxl') as writer:

    overview.to_excel(writer, sheet_name='overview')

    overview_weighted.to_excel(writer, sheet_name='overview_weighted')
    
    detail.to_excel(writer, sheet_name='detail')
    #auto_adjust_xlsx_column_width(overall, writer, sheet_name="overall", margin=5)
    
    ww_asia_diff_DPQE.to_excel(writer, sheet_name='ww_asia_diff_DPQE')
    auto_adjust_xlsx_column_width(ww_asia_diff_DPQE, writer, sheet_name="ww_asia_diff_DPQE", margin=5)
    
    projects_wo_DPQE.to_excel(writer, sheet_name='projects_wo_DPQE')
    auto_adjust_xlsx_column_width(projects_wo_DPQE, writer, sheet_name="projects_wo_DPQE", margin=5)
    
    multi_dpqe.to_excel(writer, sheet_name='multi_dpqe')
    auto_adjust_xlsx_column_width(multi_dpqe, writer, sheet_name="multi_dpqe", margin=5)

    MPdate_from_now_count.to_excel(writer, sheet_name='MPdate_from_now_count')
    auto_adjust_xlsx_column_width(MPdate_from_now_count, writer, sheet_name="MPdate_from_now_count", margin=3)
    
    MPdate_from_now.to_excel(writer, sheet_name='MPdate_from_now')
    auto_adjust_xlsx_column_width(MPdate_from_now, writer, sheet_name="MPdate_from_now", margin=3)
    
    No_MPdate_count.to_excel(writer, sheet_name='No_MPdate_count')
    auto_adjust_xlsx_column_width(No_MPdate_count, writer, sheet_name="No_MPdate_count", margin=3)
    
    No_MPdate.to_excel(writer, sheet_name='No_MPdate')
    auto_adjust_xlsx_column_width(No_MPdate, writer, sheet_name="No_MPdate", margin=3)

    Aviation_PRweb.to_excel(writer, sheet_name='Aviation_PRweb')
    auto_adjust_xlsx_column_width(Aviation_PRweb, writer, sheet_name="Aviation_PRweb", margin=3)

